# RAG using Llamaindex


In [ ]:
### Setup environment 

!python3 -m venv rag-pipeline -- quiet
!source rag/bin/activate --quiet

##### Install dependencies

!pip install -r requirements.txt --quiet

### Loader

In [2]:
from pathlib import Path

from llama_index.readers.file import PyMuPDFReader

loader = PyMuPDFReader()
docs0 = loader.load_data(file_path=Path("data/State of AI Report 2023.pdf"), metadata=True)

In [3]:
print(f" docs is a {type(docs0)}, of length {len(docs0)}, where each element is a {type(docs0[0])} object")

 docs is a <class 'list'>, of length 163, where each element is a <class 'llama_index.core.schema.Document'> object


In [4]:
print ([k for k, v in docs0[94]])

print (docs0[94].get_content())

[v for k,v in docs0[94] if k=='metadata']

docs0[94]

['id_', 'embedding', 'metadata', 'excluded_embed_metadata_keys', 'excluded_llm_metadata_keys', 'relationships', 'text', 'start_char_idx', 'end_char_idx', 'text_template', 'metadata_template', 'metadata_seperator']
    In Oct 2022, Shutterstock - a leading stock multimedia provider - announced it will work with OpenAI to bring 
DALL·E-powered content onto the platform. Then in July 2023, the two companies signed a 6-year content 
licensing agreement that would give OpenAI access to Shutterstock's image, video and music libraries and 
associated metadata for model training. Furthermore, Shutterstock will offer its customers indemniﬁcation for AI 
image creation. The company also entered into a content license with Meta for GenAI. This pro-GenAI stance is in 
stark contrast to Shutterstock’s competitor, Getty Images, which is profoundly against GenAI as evidenced by its 
ongoing lawsuit against Stability AI for copyright infringement ﬁled in Feb 2023. 
stateof.ai 2023
#stateofai | 95
 Int

Document(id_='d0ac5c48-c85e-41d6-b09e-2adf6e7af9a5', embedding=None, metadata={'total_pages': 163, 'file_path': 'data/State of AI Report 2023.pdf', 'source': '95'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="    In Oct 2022, Shutterstock - a leading stock multimedia provider - announced it will work with OpenAI to bring \nDALL·E-powered content onto the platform. Then in July 2023, the two companies signed a 6-year content \nlicensing agreement that would give OpenAI access to Shutterstock's image, video and music libraries and \nassociated metadata for model training. Furthermore, Shutterstock will offer its customers indemniﬁcation for AI \nimage creation. The company also entered into a content license with Meta for GenAI. This pro-GenAI stance is in \nstark contrast to Shutterstock’s competitor, Getty Images, which is profoundly against GenAI as evidenced by its \nongoing lawsuit against Stability AI for copyright infringement ﬁled in Fe

## Clean text and add metadata

In [5]:
import re

def clean_slide_text(text:str) -> str: 
    """
    Cleans the provided slide by removing specific patterns and extra whitespace. 
    
    Parameters:

    Returns: 
    """
    # Remove the footer text
    text = text.replace("stateof.ai 2023", "")

    # Remove the header text
    text = text.replace("Introduction  | Research  | Industry  | Politics  | Safety  | Predictions", "")

    # Remove the pattern "#stateofai | n"
    text = re.sub(r"#stateofai(\s*\|\s*\d+)?", "", text)

    # Replace multiple consecutive spaces with a single space
    text = re.sub(r" +", " ", text)

    # Remove any leading or trailing whitespace
    text = text.strip()

    return text

In [6]:
def assign_section(document):
    """
    Assigns a section to the document based on its page number.

    The function updates the 'metadata' attribute of the document with a key 'section'
    that has a value corresponding to the section the page number falls into.

    Sections:
    - Page 1 through 10: Introduction
    - Page 11 through 68: Research
    - Page 69 through 120: Politics
    - Page 121 through 137: Safety
    - Pages 138 and beyond: Predictions

    Args:
    - document (Document): The Document object to be updated.

    Returns:
    None. The function updates the Document object in-place.
    """

    page_number = int(document.metadata['source'])

    if 1 <= page_number <= 10:
        document.metadata['section'] = 'Introduction'
    elif 11 <= page_number <= 68:
        document.metadata['section'] = 'Research'
    elif 69 <= page_number <= 120:
        document.metadata['section'] = 'Politics'
    elif 121 <= page_number <= 137:
        document.metadata['section'] = 'Safety'
    else:
        document.metadata['section'] = 'Predictions'

In [7]:
# Iterate through each Document object in docs0
for doc in docs0:
    # Update the metadata using assign_section
    assign_section(doc)

    # Metadata keys that are excluded from text for the embed model.
    doc.excluded_embed_metadata_keys=['file_name']

    # Apply clean_slide_text to the text attribute1
    doc.text = clean_slide_text(doc.text)

In [8]:
docs0[94].metadata

{'total_pages': 163,
 'file_path': 'data/State of AI Report 2023.pdf',
 'source': '95',
 'section': 'Politics'}

Two options here: 
1. Directly send the entire Document object to the index
    - Maintains entire document as a single unit 
    - Useful when documents are relatively short and contexts between different parts of the document is important 
2. Covert the Document into Node objects before sending them to the index
    - Practical when the documents are long and require breaking down into chunks (or nodes) before indexing
    - Useful to retrieve specific parts of a document than the entire document

## Convert Document object to Node

- A Node represents a chunk of a source document 
- Node contain metadata and relationship information with other nodes
- Nodes are first-class citizens in LlamaIndex, this means Nodes and their attributes can be defined directly
- Every node derived from a Document will inherit the same metadata from that Document 

**Chunk Size:** 

Choosing the optimal chunk_size provides optimal results 
- Smaller chunk_size provides granular chunks, but we risk that the essential information might not be be among the top retrived chunks
- Larger chunk size might contain all necessary infromation within the top chunks 
- Increase in chunk size directs more information into the LLM. This ensures a comprehensive context but might slow down the system. 


In [9]:
import re

# Define the pattern for bullet points and newlines
split_pattern = r"\n●|\n-|\n"

# Initialize lists to store the word counts of all chunks and entire texts across all documents
chunk_word_counts = []
entire_text_word_counts = []

# Initialize a dictionary to store word counts and slide counts by section
section_data = {}

# Iterate through each Document object in your list of documents
for doc in docs0:
    # Split the document's text into chunks based on the pattern
    chunks = re.split(split_pattern, doc.text)

    # Calculate the number of words in each chunk and store it
    chunk_word_counts.extend([len(chunk.split()) for chunk in chunks])

    # Calculate the number of words in the entire text and store it
    entire_word_count = len(doc.text.split())
    entire_text_word_counts.append(entire_word_count)

    # Update the word count and slide count for the section in the dictionary
    section = doc.metadata['section']
    if section in section_data:
        section_data[section]['word_count'] += entire_word_count
        section_data[section]['slide_count'] += 1
    else:
        section_data[section] = {'word_count': entire_word_count, 'slide_count': 1}

# Calculate the total word count across all sections
total_word_count = sum(data['word_count'] for data in section_data.values())

# Calculate the number of sections
num_sections = len(section_data)

# Calculate the average word count across all sections
average_word_count_across_sections = total_word_count / num_sections

# Calculate summary statistics for chunks
average_chunk_word_count = sum(chunk_word_counts) / len(chunk_word_counts)
max_chunk_word_count = max(chunk_word_counts)

# Calculate average word count for entire texts
average_entire_text_word_count = sum(entire_text_word_counts) / len(entire_text_word_counts)

print(f"Average word count for a slide: {average_entire_text_word_count}")
print(f"Average word count per bullet point: {average_chunk_word_count}")
print(f"Longest bullet point: {max_chunk_word_count}")
print(f"Average word count in a section: {average_word_count_across_sections:.2f}")

Average word count for a slide: 138.98773006134968
Average word count per bullet point: 9.577844311377245
Longest bullet point: 28
Average word count in a section: 4531.00
